In [7]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *
from scripts.sm_utils import *

In [8]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [10]:
meta_file = '../config.tsv'
meta_file_2 = '../snakemake/map_personal/config.tsv'
genomes_file = '../snakemake/map_personal/genomes_config.tsv'

df = parse_config(meta_file)
df2 = pd.read_csv(meta_file_2, sep='\t')
df2['tech_rep'] = df2.cell_line_id+'_1'

# TODO test
# df2 = df2.loc[df2.cell_line_id == 'GM24385']

# get the genomes to download
g_df = pd.read_csv(genomes_file, sep='\t')

# maternal haplotypes
g_df['aws_mat_link'] = g_df['hap2_aws_fasta']
g_df = g_df.loc[g_df['aws_mat_link'].notnull()]
assert len(g_df.loc[g_df['aws_mat_link'].str.contains('maternal')].index) == len(g_df.index)

# paternal haplotypes
g_df['aws_pat_link'] = g_df['hap1_aws_fasta']
g_df = g_df.loc[g_df['aws_pat_link'].notnull()]
assert len(g_df.loc[g_df['aws_pat_link'].str.contains('paternal')].index) == len(g_df.index)


genome_cols = ['same_population_sample', 'european_sample',	'afr_sample']
g_df = g_df.loc[(g_df['sample'].isin(df2[genome_cols[0]]))|
                (g_df['sample'].isin(df2[genome_cols[1]]))|
                (g_df['sample'].isin(df2[genome_cols[2]]))]

# a little more df2 formatting
df2 = df2[['tech_rep', 'same_population_sample',
           'european_sample', 'afr_sample']].melt(id_vars='tech_rep')
df2 = df2.reset_index()
df2 = df2.rename({'variable':'assembly_status',
                  'value': 'assembly_sample'},
                  axis=1)

# limit just to the samples where we'll do this
df = df.loc[df.tech_rep.isin(df2.tech_rep.tolist())]

# get a key for assembly status, assembly sample, and actual sample
df2['dataset_key'] = df2.assembly_status+'_'+\
                     df2.assembly_sample+'_'+\
                     df2.tech_rep


In [17]:
sample = 'HG00621_1' # params.sample
sample_1 = sample.split('_')[0]
assemblies = ['same_population_sample',
              'european_sample',
              'afr_sample']
meta = load_meta()
pop = meta.loc[meta['sample'] == sample_1, 'population'].values[0]
c_dict, _ = get_population_colors()
color = c_dict[pop]

# TODO 
df2 = df2.loc[df2.tech_rep==sample]

In [20]:
expand(proc_cfg(config['lr']['map_personal']['bam_mapqs'],od),
       zip,
       assembly_status=df2.assembly_status.tolist(),
       assembly_sample=df2.assembly_sample.tolist(),
       sample=df2.tech_rep.tolist())

['../data/map_personal/same_population_sample/HG00673_HG00621_1_mapqs.txt',
 '../data/map_personal/european_sample/HG002_HG00621_1_mapqs.txt',
 '../data/map_personal/afr_sample/HG02717_HG00621_1_mapqs.txt']